# Checking the tracking on the manually segmented macrophages

#### Load modules

In [4]:
import os
from glob import glob
from skimage import io
import btrack
import shutil
import napari
from homuncu_loc.dataio import try_load_nemo_h5
# from tqdm.auto import tqdm
# import pandas as pd
# import matplotlib.pyplot as plt

### Pre-defined images to pay attention to

In [5]:
# Create a list called image_IDs
image_IDs = [1541,
1544,
1550,
1551,
1554,
1556,
1557,
1558,
1568]

# Print the value of image_IDs
image_IDs

[1541, 1544, 1550, 1551, 1554, 1556, 1557, 1558, 1568]

### Set address for NEMO

In [6]:
base_dir = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm'

## Find corresponding image filenames

In [7]:
# Set the base directory of the images
image_basedir = os.path.join(base_dir, 'home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/images')

# Create a list of image filenames using list comprehension
# Each filename is obtained by using glob function to find the image file that matches the image_ID in the given directory
image_fns = [glob(os.path.join(image_basedir, f'*{image_ID}*.tif'))[0] for image_ID in image_IDs]

# Print the list of image filenames
image_fns

['/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/images/20231127_20X_23-03-072A6_DAPI_ZO-1_CD16_Mtb_ATG14KO_48hpi_Mult...nel Z-Stack_20231127_1541.tif',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/images/20231127_20X_23-03-072A6_DAPI_ZO-1_CD16_Mtb_ATG14KO_48hpi_Mult...nel Z-Stack_20231127_1544.tif',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/images/20231127_20X_23-03-072A5_DAPI_ZO-1_CD16_Mtb_ATG14KO_2hpi_Multichannel Z-Stack_20231127_1550.tif',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homunc

## Find measurement filenames for these images

In [8]:
# Set the base directory of the single-cell analyses
sc_basedir = os.path.join(base_dir, 'home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses')

# Create a list of single-cell analysis filenames using list comprehension
# Each filename is obtained by using glob function to find the file that matches the image_ID in the given directory
sc_fns = [glob(os.path.join(sc_basedir, f'*{image_ID}*gt_mphi.h5'))[0] for image_ID in image_IDs]

# Print the list of single-cell analysis filenames
sc_fns

['/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20231127_20X_23-03-072A6_DAPI_ZO-1_CD16_Mtb_ATG14KO_48hpi_Mult...nel Z-Stack_20231127_1541_gt_mphi.h5',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20231127_20X_23-03-072A6_DAPI_ZO-1_CD16_Mtb_ATG14KO_48hpi_Mult...nel Z-Stack_20231127_1544_gt_mphi.h5',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20231127_20X_23-03-072A5_DAPI_ZO-1_CD16_Mtb_ATG14KO_2hpi_Multichannel Z-Stack_20231127_1550_gt_mphi.h5',
 '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutier

### Iteratively load and check images

In [ ]:
# Iterate through each ID in the image_IDs list
for ID in image_IDs:
    
    # Find the image filename that matches the ID
    image_fn = [fn for fn in image_fns if str(ID) in fn][0]
    
    # Find the single-cell analysis filename that matches the ID
    sc_fn = [fn for fn in sc_fns if str(ID) in fn][0]

    try:
        # Attempt to open the single-cell analysis file for reading
        with btrack.io.HDF5FileHandler(sc_fn, 'r', obj_type='obj_type_1') as reader:
            # Retrieve segmentation masks and tracks from the file
            masks = reader.segmentation
            tracks = reader.tracks

    except Exception as e:
        # Handle exceptions that occur when opening the file (e.g., file not found, access issues)
        print(f"Failed to load from server location: {e}. Attempting to copy file locally.")
        
        # Copy the single-cell analysis file to a temporary local folder for easier access
        temp_sc_dir = './temp_sc_dir'
        os.makedirs(temp_sc_dir, exist_ok=True)  # Create the directory if it doesn't exist
        local_sc_fn = os.path.join(temp_sc_dir, os.path.basename(sc_fn))  # Construct the local file path
        shutil.copy(sc_fn, local_sc_fn)  # Perform the file copy
        print(f"Copied {sc_fn} to {local_sc_fn}")

        try:
            # Try opening the file again, this time from the local copy
            with btrack.io.HDF5FileHandler(local_sc_fn, 'r', obj_type='obj_type_1') as reader:
                masks = reader.segmentation
                tracks = reader.tracks
            print("Loaded file from local copy successfully.")
        except Exception as e:
            # Handle exceptions that occur when opening the local copy
            print(f"Failed to load file from local copy: {e}")
            continue  # Skip further processing and move to the next ID

    # Read the image file associated with the current ID
    images = io.imread(image_fn)

    # Convert the tracks to a format compatible with Napari
    napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim=2)
    
    # Initialize a Napari viewer with the title as the base name of the image file
    viewer = napari.Viewer(title=os.path.basename(image_fn))
    viewer.add_image(images, channel_axis=-1)  # Add the image to the viewer, assuming it has multiple channels
    viewer.add_labels(masks)  # Add the segmentation masks as labels
    viewer.add_tracks(napari_tracks)  # Add the tracks to the viewer
    
    # Display the Napari viewer
    viewer.show(block=True)

[INFO][2023/12/11 05:22:09 PM] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20231127_20X_23-03-072A6_DAPI_ZO-1_CD16_Mtb_ATG14KO_48hpi_Mult...nel Z-Stack_20231127_1541_gt_mphi.h5...
[INFO][2023/12/11 05:22:13 PM] Loading segmentation (155, 2304, 2304)
[INFO][2023/12/11 05:22:13 PM] Loading tracks/obj_type_1
[INFO][2023/12/11 05:22:13 PM] Loading LBEP/obj_type_1
[INFO][2023/12/11 05:22:13 PM] Loading objects/obj_type_1 (1064, 5) (1064 filtered: None)
[INFO][2023/12/11 05:22:13 PM] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/homuncu_loc_image_analysis/iAT1_iAT2_iVEC_macrophage_experiments/DAPI_ZO1_CD16_MTB/sc_analyses/20231127_20X_23-03-072A6_DAPI_ZO-1_CD16_Mtb_ATG14KO_48hpi_Mult...nel Z-Stack_20231127_1541_gt_mphi.h5
TiffPage 0: TypeError